In [195]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score


In [196]:
data_sucio = pd.read_csv('DataAnalytics.csv')
data_sucio.head(5)

,Administrador,Usuario,botón correcto,tiempo de interacción,mini juego,número de interacción,color presionado,dificultad,fecha,Juego,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,1.0,5.399169,Despegue,1.0,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,0.0,0.0,0.0
1,nicolas,nicolas,0.0,1.283400,Despegue,2.0,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,0.0,0.0
2,nicolas,nicolas,1.0,2.700226,Despegue,3.0,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,0.0,0.0,0.0
3,nicolas,nicolas,0.0,3.050262,Despegue,4.0,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,0.0,0.0
4,nicolas,nicolas,0.0,4.750256,Despegue,5.0,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,0.0,0.0


In [197]:
data_sucio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Administrador          5865 non-null   object 
 1   Usuario                5865 non-null   object 
 2   botón correcto         5103 non-null   float64
 3   tiempo de interacción  5103 non-null   float64
 4   mini juego             5709 non-null   object 
 5   número de interacción  5103 non-null   float64
 6   color presionado       5103 non-null   object 
 7   dificultad             5865 non-null   object 
 8   fecha                  5865 non-null   object 
 9   Juego                  5865 non-null   object 
 10  auto push              5103 non-null   float64
 11  tiempo de lección      5688 non-null   float64
 12  tiempo de sesión       5259 non-null   float64
dtypes: float64(6), object(7)
memory usage: 595.8+ KB


## Limpieza de Nulos 

In [198]:
data_cuanti=data_sucio.select_dtypes(include=["float64","int64","float","int"])
data_cuali=data_sucio.select_dtypes(include=["object","datetime","category","datetime64[ns]"])
data_cuali.info()
data_cuanti.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Administrador     5865 non-null   object
 1   Usuario           5865 non-null   object
 2   mini juego        5709 non-null   object
 3   color presionado  5103 non-null   object
 4   dificultad        5865 non-null   object
 5   fecha             5865 non-null   object
 6   Juego             5865 non-null   object
dtypes: object(7)
memory usage: 320.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   botón correcto         5103 non-null   float64
 1   tiempo de interacción  5103 non-null   float64
 2   número de interacción  5103 non-null   float64
 3   auto push              5103 non-null   float64
 4   tiempo de lecci

In [199]:
cualis_bfill=data_cuali.fillna(method="bfill")
cualis_ffill=data_cuali.fillna(method="ffill")
cuantias_mean=data_cuanti.fillna(round(data_cuanti.mean(),1))

data_sin_nulos = pd.concat([cuantias_mean,cualis_ffill],axis=1)

data_sin_nulos.head(5)

C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3586926086.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_bfill=data_cuali.fillna(method="bfill")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3586926086.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_ffill=data_cuali.fillna(method="ffill")


,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego
0,1.0,5.399169,1.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro
1,0.0,1.283400,2.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro
2,1.0,2.700226,3.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro
3,0.0,3.050262,4.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro
4,0.0,4.750256,5.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro


In [200]:
data=data_sin_nulos.isnull().sum().sum()
print(data)

0


In [201]:
DATA = data

## Limpieza Outliers

In [202]:
cuantitativas=data_sucio.select_dtypes(include=["float64","int64","float","int"])
cualitativas=data_sucio.select_dtypes(include=["object","datetime","category"])
y=cuantitativas

percentile25=y.quantile(0.25)
percentile75=y.quantile(0.75)
iqr= percentile75-percentile25

Limite_Superior_iqr= percentile75+1.5*iqr
Limite_Inferior_iqr= percentile25-1.5*iqr
iqr=cuantitativas[(y<=Limite_Superior_iqr)&y>=(Limite_Inferior_iqr)]
iqr2=iqr.fillna(round(iqr.mean(),1))
rango_intercuartilico=pd.concat([cualitativas,iqr2],axis=1)
rango_intercuartilico.head(5)

,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,1.0,5.399169,1.0,0.0,0.0,0.0
1,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,1.283400,2.0,0.0,0.0,0.0
2,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,1.0,2.700226,3.0,0.0,0.0,0.0
3,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,3.050262,4.0,0.0,0.0,0.0
4,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,4.750256,5.0,0.0,0.0,0.0


In [203]:
cualis_bfill=data_cuali.fillna(method="bfill")
cualis_ffill=data_cuali.fillna(method="ffill")
cuantias_mean=data_cuanti.fillna(round(data_cuanti.mean(),1))

data = pd.concat([cuantias_mean,cualis_ffill],axis=1)

data.head(5)

C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3308866133.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_bfill=data_cuali.fillna(method="bfill")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\3308866133.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cualis_ffill=data_cuali.fillna(method="ffill")


,botón correcto,tiempo de interacción,número de interacción,auto push,tiempo de lección,tiempo de sesión,Administrador,Usuario,mini juego,color presionado,dificultad,fecha,Juego
0,1.0,5.399169,1.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro
1,0.0,1.283400,2.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro
2,1.0,2.700226,3.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro
3,0.0,3.050262,4.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro
4,0.0,4.750256,5.0,0.0,0.0,0.0,nicolas,nicolas,Despegue,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro


In [204]:
outliers_limpios=data.isnull().sum().sum()
print(outliers_limpios)

0


## Analizar la correlación logística (botón correcto, juego, auto push)

In [205]:
df = pd.read_csv('DataAnalytics.csv')
df.head()

,Administrador,Usuario,botón correcto,tiempo de interacción,mini juego,número de interacción,color presionado,dificultad,fecha,Juego,auto push,tiempo de lección,tiempo de sesión
0,nicolas,nicolas,1.0,5.399169,Despegue,1.0,blue,Episodio 1,25/01/2024 09:26:42 a. m.,Astro,0.0,0.0,0.0
1,nicolas,nicolas,0.0,1.283400,Despegue,2.0,violet,Episodio 1,25/01/2024 09:26:46 a. m.,Astro,0.0,0.0,0.0
2,nicolas,nicolas,1.0,2.700226,Despegue,3.0,green,Episodio 1,25/01/2024 09:26:48 a. m.,Astro,0.0,0.0,0.0
3,nicolas,nicolas,0.0,3.050262,Despegue,4.0,green,Episodio 1,25/01/2024 09:26:57 a. m.,Astro,0.0,0.0,0.0
4,nicolas,nicolas,0.0,4.750256,Despegue,5.0,green,Episodio 1,25/01/2024 09:26:58 a. m.,Astro,0.0,0.0,0.0


In [206]:
# Crear un diccionario para almacenar los modelos de regresión logística
logistic_models = {}

In [207]:
# Preparamos los datos, eliminando nulos y codificando 'Juego'
df_clean = df[['botón correcto', 'auto push', 'Juego', 'número de interacción', 'tiempo de interacción']].dropna()

In [208]:
# Convertir 'Juego' en variable binaria para regresión logística (0 si es el juego más común, 1 si no)
juego_mas_comun = df_clean['Juego'].mode()[0]
df_clean['Juego_binaria'] = (df_clean['Juego'] != juego_mas_comun).astype(int)

In [209]:
df_clean['Juego_binaria']

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
21      0
22      0
23      0
24      0
25      0
26      0
28      0
29      0
30      0
31      0
32      0
33      0
34      0
35      0
36      0
37      0
38      0
39      0
41      0
42      0
43      0
44      0
45      0
46      0
47      0
48      0
49      0
51      0
52      0
53      0
54      0
55      0
56      0
57      0
58      0
59      0
60      0
61      0
62      0
64      0
65      0
66      0
67      0
68      0
69      0
70      0
71      0
72      0
73      0
74      0
75      0
76      0
77      0
78      0
79      0
81      0
82      0
83      0
84      0
85      0
86      0
87      0
88      0
89      0
90      0
91      0
92      0
93      0
94      0
95      0
96      0
97      0
98      0
99      0
100     0
101     0
102     0
103     0
104     0
105     0
106     0


In [210]:
df_clean.head()

,botón correcto,auto push,Juego,número de interacción,tiempo de interacción,Juego_binaria
0,1.0,0.0,Astro,1.0,5.399169,0
1,0.0,0.0,Astro,2.0,1.283400,0
2,1.0,0.0,Astro,3.0,2.700226,0
3,0.0,0.0,Astro,4.0,3.050262,0
4,0.0,0.0,Astro,5.0,4.750256,0


In [211]:
Var_Indepx= df_clean[['número de interacción', 'tiempo de interacción']]
Var_Depy1=df_clean['botón correcto']
Var_Depy2=df_clean['auto push']
Var_Depy3=df_clean['Juego_binaria']

In [212]:
#Redefinimos las variables 
X= Var_Indepx
y1= Var_Depy1
y2= Var_Depy2
y3= Var_Depy3

In [213]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size=0.3, random_state =None)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y1, test_size=0.3, random_state =None)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y1, test_size=0.3, random_state =None)

In [214]:
#Se escalan todos los datos
escalar = StandardScaler()

In [215]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)

X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)

In [216]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [217]:
#Entrenamos el modelo
algoritmo.fit(X_train1, y_train1)
algoritmo.fit(X_train2, y_train2)
algoritmo.fit(X_train3, y_train3)

LogisticRegression()

In [218]:
#Realizamos una predicción
y_pred1 = algoritmo.predict(X_test1) 
y_pred1

y_pred2 = algoritmo.predict(X_test2) 
y_pred2

y_pred3 = algoritmo.predict(X_test3) 
y_pred3

array([1., 1., 1., ..., 0., 1., 1.], shape=(1531,))

In [219]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz1 = confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión 1: ')
print(matriz1)

matriz2 = confusion_matrix(y_test2, y_pred2)
print('Matriz de Confusión 2: ')
print(matriz2)

matriz3 = confusion_matrix(y_test3, y_pred3)
print('Matriz de Confusión 3: ')
print(matriz3)

Matriz de Confusión 1: 
[[392 394]
 [133 612]]
Matriz de Confusión 2: 
[[374 393]
 [141 623]]
Matriz de Confusión 3: 
[[403 360]
 [139 629]]


In [220]:
# Modelo 1: botón correcto ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 1: Botón Correcto 0.0")
precision1_0= precision_score (y_test1, y_pred1, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision1_0 )
exactitud1_0 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo: ', exactitud1_0)
sensibilidad1_0 = recall_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad1_0)
puntajef1_1_0 = f1_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_1_0)
print("\n")

print("Modelo 1: Botón correcto 1.0 ")
precision1= precision_score (y_test1, y_pred1, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision1 )
exactitud1 = accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo: ', exactitud1)
sensibilidad1 = recall_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad1)
puntajef1_1 = f1_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_1)

Modelo 1: Botón Correcto 0.0
Precisión del modelo:  0.7466666666666667
Exactitud del modelo:  0.6557805355976486
Sensibilidad del modelo:  0.49872773536895676
Puntaje F1 del modelo:  0.5980167810831426


Modelo 1: Botón correcto 1.0 
Precisión del modelo:  0.6083499005964215
Exactitud del modelo:  0.6557805355976486
Sensibilidad del modelo:  0.8214765100671141
Puntaje F1 del modelo:  0.6990291262135923


In [221]:
# Modelo 2: Auto Push ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 2: Auto push 0.0")
precision2_0= precision_score (y_test2, y_pred2, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision2_0 )
exactitud2_0 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo: ', exactitud2_0)
sensibilidad2_0 = recall_score(y_test2, y_pred2, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad2_0)
puntajef1_2_0 = f1_score(y_test2, y_pred2, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_2_0)
print("\n")


print("Modelo 2: Auto push 1.0")
precision2= precision_score (y_test2, y_pred2, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision2 )
exactitud2 = accuracy_score(y_test2, y_pred2)
print('Exactitud del modelo: ', exactitud2)
sensibilidad2 = recall_score(y_test2, y_pred2, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad2)
puntajef1_2 = f1_score(y_test2, y_pred2, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_2)

Modelo 2: Auto push 0.0
Precisión del modelo:  0.7262135922330097
Exactitud del modelo:  0.651208360548661
Sensibilidad del modelo:  0.4876140808344198
Puntaje F1 del modelo:  0.5834633385335414


Modelo 2: Auto push 1.0
Precisión del modelo:  0.6131889763779528
Exactitud del modelo:  0.651208360548661
Sensibilidad del modelo:  0.8154450261780105
Puntaje F1 del modelo:  0.7


In [222]:
# Modelo 3: Juego ~ número de interacción + tiempo de interacción
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("Modelo 3: Juego 0.0")
precision3_0= precision_score (y_test3, y_pred3, average="binary", pos_label=0.0)
print('Precisión del modelo: ',precision3_0 )
exactitud3_0 = accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo: ', exactitud3_0)
sensibilidad3_0 = recall_score(y_test3, y_pred3, average="binary", pos_label=0.0)
print('Sensibilidad del modelo: ', sensibilidad3_0)
puntajef1_3_0 = f1_score(y_test1, y_pred1, average="binary", pos_label=0.0)
print('Puntaje F1 del modelo: ', puntajef1_3_0)
print("\n")

print("Modelo 3: Juego 1.0")
precision3= precision_score (y_test3, y_pred3, average="binary", pos_label=1.0)
print('Precisión del modelo: ',precision3 )
exactitud3 = accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo: ', exactitud3)
sensibilidad3 = recall_score(y_test3, y_pred3, average="binary", pos_label=1.0)
print('Sensibilidad del modelo: ', sensibilidad3)
puntajef1_3 = f1_score(y_test1, y_pred1, average="binary", pos_label=1.0)
print('Puntaje F1 del modelo: ', puntajef1_3)

Modelo 3: Juego 0.0
Precisión del modelo:  0.7435424354243543
Exactitud del modelo:  0.674069235793599
Sensibilidad del modelo:  0.528178243774574
Puntaje F1 del modelo:  0.5980167810831426


Modelo 3: Juego 1.0
Precisión del modelo:  0.6359959555106168
Exactitud del modelo:  0.674069235793599
Sensibilidad del modelo:  0.8190104166666666
Puntaje F1 del modelo:  0.6990291262135923


## Convertir las variables en tipo dicotómica y analizarlas

Función de Regresión Logística

In [223]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def regresioncion_logistica(dicotomica,si,no):
    dicotomica=dicotomica.replace(si,"0")
    dicotomica=dicotomica.replace(no,"1")
    dicotomica = dicotomica.apply(pd.to_numeric, errors="coerce")
    Var_Indepx= data[['número de interacción', 'tiempo de interacción']]
    Var_Depy=dicotomica
    X= Var_Indepx
    Y= Var_Depy
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state =None)
    escalar = StandardScaler()
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.transform(X_test)
    algoritmo = LogisticRegression()
    algoritmo.fit(X_train, Y_train)
    Y_pred = algoritmo.predict(X_test) 
    matriz = confusion_matrix(Y_test, Y_pred)
    print('Matriz de Confusión: ')
    print(matriz)
    print("Modelo 0:")
    precision0= precision_score (Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Precisión del modelo: ',precision0 )
    exactitud0 = accuracy_score(Y_test, Y_pred)
    print('Exactitud del modelo: ', exactitud0)
    sensibilidad0 = recall_score(Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Sensibilidad del modelo: ', sensibilidad0)
    puntajef0 = f1_score(Y_test, Y_pred, average="binary", pos_label=0.0)
    print('Puntaje F1 del modelo: ', puntajef0)
    print("\n")
    print("Modelo 1:")
    precision1= precision_score (Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Precisión del modelo: ',precision1 )
    exactitud1 = accuracy_score(Y_test, Y_pred)
    print('Exactitud del modelo: ', exactitud1)
    sensibilidad1 = recall_score(Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Sensibilidad del modelo: ', sensibilidad1)
    puntajef1_1 = f1_score(Y_test, Y_pred, average="binary", pos_label=1.0)
    print('Puntaje F1 del modelo: ', puntajef1_1)

Color Presionado (5)

In [224]:
copr = np.unique(data_sin_nulos['color presionado'])
copr

array(['blue', 'green', 'red', 'violet', 'yellow'], dtype=object)

In [225]:
blue= data_sin_nulos['color presionado'].replace(["green", "red", "violet", "yellow"], "no blue")
blue

0          blue
1       no blue
2       no blue
3       no blue
4       no blue
5       no blue
6       no blue
7       no blue
8       no blue
9          blue
10      no blue
11      no blue
12      no blue
13         blue
14         blue
15      no blue
16      no blue
17         blue
18         blue
19      no blue
20      no blue
21      no blue
22      no blue
23      no blue
24      no blue
25      no blue
26      no blue
27      no blue
28      no blue
29      no blue
30         blue
31      no blue
32      no blue
33      no blue
34      no blue
35      no blue
36      no blue
37      no blue
38      no blue
39         blue
40         blue
41      no blue
42      no blue
43      no blue
44      no blue
45      no blue
46         blue
47      no blue
48      no blue
49      no blue
50      no blue
51      no blue
52      no blue
53      no blue
54      no blue
55      no blue
56      no blue
57      no blue
58      no blue
59      no blue
60      no blue
61         blue
62      

In [226]:
blue_relo=regresioncion_logistica(blue,"blue","no blue")

Matriz de Confusión: 
[[   0  371]
 [   0 1389]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7892045454545454
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7892045454545454
Exactitud del modelo:  0.7892045454545454
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8821848205779612


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [227]:
green= data_sin_nulos['color presionado'].replace(["blue", "red", "violet", "yellow"], "no green")
green

0       no green
1       no green
2          green
3          green
4          green
5          green
6          green
7       no green
8          green
9       no green
10         green
11      no green
12      no green
13      no green
14      no green
15      no green
16      no green
17      no green
18      no green
19      no green
20      no green
21      no green
22      no green
23      no green
24      no green
25      no green
26      no green
27      no green
28      no green
29      no green
30      no green
31         green
32      no green
33      no green
34         green
35      no green
36         green
37         green
38         green
39      no green
40      no green
41      no green
42      no green
43      no green
44      no green
45      no green
46      no green
47      no green
48      no green
49      no green
50      no green
51         green
52      no green
53      no green
54      no green
55      no green
56      no green
57      no green
58      no gre

In [228]:
green_relo=regresioncion_logistica(green,"green","no green")

Matriz de Confusión: 
[[   0  439]
 [   0 1321]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7505681818181819
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7505681818181819
Exactitud del modelo:  0.7505681818181819
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.857513794222655


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [229]:
red= data_sin_nulos['color presionado'].replace(["blue", "green", "violet", "yellow"], "no red")
red

0       no red
1       no red
2       no red
3       no red
4       no red
5       no red
6       no red
7       no red
8       no red
9       no red
10      no red
11      no red
12      no red
13      no red
14      no red
15      no red
16      no red
17      no red
18      no red
19      no red
20      no red
21      no red
22      no red
23      no red
24      no red
25      no red
26      no red
27      no red
28      no red
29      no red
30      no red
31      no red
32      no red
33      no red
34      no red
35      no red
36      no red
37      no red
38      no red
39      no red
40      no red
41      no red
42      no red
43      no red
44      no red
45      no red
46      no red
47      no red
48      no red
49      no red
50      no red
51      no red
52      no red
53      no red
54      no red
55      no red
56      no red
57      no red
58      no red
59      no red
60      no red
61      no red
62      no red
63      no red
64      no red
65      no red
66      no

In [230]:
red_relo=regresioncion_logistica(red,"red","no red")

Matriz de Confusión: 
[[   0   10]
 [   0 1750]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.9943181818181818
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.9943181818181818
Exactitud del modelo:  0.9943181818181818
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9971509971509972


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [231]:
violet= data_sin_nulos['color presionado'].replace(["blue", "green", "red", "yellow"], "no violet")
violet

0       no violet
1          violet
2       no violet
3       no violet
4       no violet
5       no violet
6       no violet
7       no violet
8       no violet
9       no violet
10      no violet
11         violet
12         violet
13      no violet
14      no violet
15         violet
16         violet
17      no violet
18      no violet
19      no violet
20      no violet
21         violet
22         violet
23      no violet
24         violet
25         violet
26      no violet
27      no violet
28         violet
29         violet
30      no violet
31      no violet
32      no violet
33      no violet
34      no violet
35      no violet
36      no violet
37      no violet
38      no violet
39      no violet
40      no violet
41      no violet
42      no violet
43      no violet
44      no violet
45      no violet
46      no violet
47         violet
48         violet
49         violet
50         violet
51      no violet
52      no violet
53         violet
54         violet
55        

In [232]:
violet_relo=regresioncion_logistica(violet,"violet","no violet")

Matriz de Confusión: 
[[  70  431]
 [   3 1256]]
Modelo 0:
Precisión del modelo:  0.958904109589041
Exactitud del modelo:  0.7534090909090909
Sensibilidad del modelo:  0.13972055888223553
Puntaje F1 del modelo:  0.24390243902439024


Modelo 1:
Precisión del modelo:  0.7445168938944873
Exactitud del modelo:  0.7534090909090909
Sensibilidad del modelo:  0.9976171564733916
Puntaje F1 del modelo:  0.8526816021724372


In [233]:
yellow= data_sin_nulos['color presionado'].replace(["blue", "green", "red", "violet"], "no yellow")
yellow

0       no yellow
1       no yellow
2       no yellow
3       no yellow
4       no yellow
5       no yellow
6       no yellow
7          yellow
8       no yellow
9       no yellow
10      no yellow
11      no yellow
12      no yellow
13      no yellow
14      no yellow
15      no yellow
16      no yellow
17      no yellow
18      no yellow
19         yellow
20         yellow
21      no yellow
22      no yellow
23         yellow
24      no yellow
25      no yellow
26         yellow
27         yellow
28      no yellow
29      no yellow
30      no yellow
31      no yellow
32         yellow
33         yellow
34      no yellow
35         yellow
36      no yellow
37      no yellow
38      no yellow
39      no yellow
40      no yellow
41         yellow
42         yellow
43         yellow
44         yellow
45         yellow
46      no yellow
47      no yellow
48      no yellow
49      no yellow
50      no yellow
51      no yellow
52         yellow
53      no yellow
54      no yellow
55      no

In [234]:
yellow_relo=regresioncion_logistica(yellow,"yellow","no yellow")

Matriz de Confusión: 
[[   0  429]
 [   0 1331]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.75625
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.75625
Exactitud del modelo:  0.75625
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8612099644128114


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dificultad

In [235]:
dif = np.unique(data_sin_nulos['dificultad'])
dif

array(['Episodio 1', 'Episodio 2', 'Episodio 3', 'Episodio 4'],
      dtype=object)

In [236]:
E1= data_sin_nulos['dificultad'].replace(["Episodio 2", "Episodio 3", "Episodio 4"], "NO Episodio 1")
E1

0          Episodio 1
1          Episodio 1
2          Episodio 1
3          Episodio 1
4          Episodio 1
5          Episodio 1
6          Episodio 1
7          Episodio 1
8          Episodio 1
9          Episodio 1
10         Episodio 1
11         Episodio 1
12         Episodio 1
13         Episodio 1
14         Episodio 1
15         Episodio 1
16         Episodio 1
17         Episodio 1
18         Episodio 1
19         Episodio 1
20         Episodio 1
21         Episodio 1
22         Episodio 1
23         Episodio 1
24         Episodio 1
25         Episodio 1
26         Episodio 1
27         Episodio 1
28         Episodio 1
29         Episodio 1
30         Episodio 1
31         Episodio 1
32         Episodio 1
33         Episodio 1
34         Episodio 1
35         Episodio 1
36         Episodio 1
37         Episodio 1
38         Episodio 1
39         Episodio 1
40         Episodio 1
41         Episodio 1
42         Episodio 1
43         Episodio 1
44         Episodio 1
45        

In [237]:
E1_relo=regresioncion_logistica(E1,"Episodio 1","NO Episodio 1")

Matriz de Confusión: 
[[ 99 642]
 [ 40 979]]
Modelo 0:
Precisión del modelo:  0.7122302158273381
Exactitud del modelo:  0.6125
Sensibilidad del modelo:  0.13360323886639677
Puntaje F1 del modelo:  0.225


Modelo 1:
Precisión del modelo:  0.6039481801357187
Exactitud del modelo:  0.6125
Sensibilidad del modelo:  0.9607458292443573
Puntaje F1 del modelo:  0.7416666666666667


In [238]:
E2= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 3", "Episodio 4"], "NO Episodio 2")
E2

0       NO Episodio 2
1       NO Episodio 2
2       NO Episodio 2
3       NO Episodio 2
4       NO Episodio 2
5       NO Episodio 2
6       NO Episodio 2
7       NO Episodio 2
8       NO Episodio 2
9       NO Episodio 2
10      NO Episodio 2
11      NO Episodio 2
12      NO Episodio 2
13      NO Episodio 2
14      NO Episodio 2
15      NO Episodio 2
16      NO Episodio 2
17      NO Episodio 2
18      NO Episodio 2
19      NO Episodio 2
20      NO Episodio 2
21      NO Episodio 2
22      NO Episodio 2
23      NO Episodio 2
24      NO Episodio 2
25      NO Episodio 2
26      NO Episodio 2
27      NO Episodio 2
28      NO Episodio 2
29      NO Episodio 2
30      NO Episodio 2
31      NO Episodio 2
32      NO Episodio 2
33      NO Episodio 2
34      NO Episodio 2
35      NO Episodio 2
36      NO Episodio 2
37      NO Episodio 2
38      NO Episodio 2
39      NO Episodio 2
40      NO Episodio 2
41      NO Episodio 2
42      NO Episodio 2
43      NO Episodio 2
44      NO Episodio 2
45      NO

In [239]:
E2_relo=regresioncion_logistica(E2,"Episodio 2","NO Episodio 2")

Matriz de Confusión: 
[[   0  463]
 [   0 1297]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7369318181818182
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7369318181818182
Exactitud del modelo:  0.7369318181818182
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.8485443245011449


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [240]:
E3= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 2", "Episodio 4"], "NO Episodio 3")
E3

0       NO Episodio 3
1       NO Episodio 3
2       NO Episodio 3
3       NO Episodio 3
4       NO Episodio 3
5       NO Episodio 3
6       NO Episodio 3
7       NO Episodio 3
8       NO Episodio 3
9       NO Episodio 3
10      NO Episodio 3
11      NO Episodio 3
12      NO Episodio 3
13      NO Episodio 3
14      NO Episodio 3
15      NO Episodio 3
16      NO Episodio 3
17      NO Episodio 3
18      NO Episodio 3
19      NO Episodio 3
20      NO Episodio 3
21      NO Episodio 3
22      NO Episodio 3
23      NO Episodio 3
24      NO Episodio 3
25      NO Episodio 3
26      NO Episodio 3
27      NO Episodio 3
28      NO Episodio 3
29      NO Episodio 3
30      NO Episodio 3
31      NO Episodio 3
32      NO Episodio 3
33      NO Episodio 3
34      NO Episodio 3
35      NO Episodio 3
36      NO Episodio 3
37      NO Episodio 3
38      NO Episodio 3
39      NO Episodio 3
40      NO Episodio 3
41      NO Episodio 3
42      NO Episodio 3
43      NO Episodio 3
44      NO Episodio 3
45      NO

In [241]:
E3_relo=regresioncion_logistica(E3,"Episodio 3","NO Episodio 3")

Matriz de Confusión: 
[[   0  379]
 [   3 1378]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.7829545454545455
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.7842914058053501
Exactitud del modelo:  0.7829545454545455
Sensibilidad del modelo:  0.997827661115134
Puntaje F1 del modelo:  0.8782664117272148


In [242]:
E4= data_sin_nulos['dificultad'].replace(["Episodio 1", "Episodio 2", "Episodio 3"], "NO Episodio 4")
E4

0       NO Episodio 4
1       NO Episodio 4
2       NO Episodio 4
3       NO Episodio 4
4       NO Episodio 4
5       NO Episodio 4
6       NO Episodio 4
7       NO Episodio 4
8       NO Episodio 4
9       NO Episodio 4
10      NO Episodio 4
11      NO Episodio 4
12      NO Episodio 4
13      NO Episodio 4
14      NO Episodio 4
15      NO Episodio 4
16      NO Episodio 4
17      NO Episodio 4
18      NO Episodio 4
19      NO Episodio 4
20      NO Episodio 4
21      NO Episodio 4
22      NO Episodio 4
23      NO Episodio 4
24      NO Episodio 4
25      NO Episodio 4
26      NO Episodio 4
27      NO Episodio 4
28      NO Episodio 4
29      NO Episodio 4
30      NO Episodio 4
31      NO Episodio 4
32      NO Episodio 4
33      NO Episodio 4
34      NO Episodio 4
35      NO Episodio 4
36      NO Episodio 4
37      NO Episodio 4
38      NO Episodio 4
39      NO Episodio 4
40      NO Episodio 4
41      NO Episodio 4
42      NO Episodio 4
43      NO Episodio 4
44      NO Episodio 4
45      NO

In [243]:
E4_relo=regresioncion_logistica(E4,"Episodio 4","NO Episodio 4")

Matriz de Confusión: 
[[   0  222]
 [   0 1538]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.8738636363636364
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.8738636363636364
Exactitud del modelo:  0.8738636363636364
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9326864766525167


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mini Juego(2)

In [244]:
#numero de interacción, usuario(5))
miju = np.unique(data_sin_nulos['mini juego'])
miju

array(['Asteroides', 'Despegue', 'Estrellas', 'Gusanos', 'MiniGame_0',
       'MiniGame_1', 'MiniGame_2', 'MiniGame_3', 'Restaurante',
       'animales y colores', 'figuras y colores', 'partes del cuerpo',
       'sonidos y animales'], dtype=object)

In [245]:
Asteroides= data_sin_nulos['mini juego'].replace(["Despegue", "Estrellas", "Gusanos", "MiniGame_0", "MiniGame_1", "MiniGame_2", "MiniGame_3", "Restaurante", "animales y colores", "figuras y colores", "partes del cuerpo", "sonidos y animales"], "NO Asteroides")
Asteroides

0       NO Asteroides
1       NO Asteroides
2       NO Asteroides
3       NO Asteroides
4       NO Asteroides
5       NO Asteroides
6       NO Asteroides
7       NO Asteroides
8       NO Asteroides
9       NO Asteroides
10      NO Asteroides
11      NO Asteroides
12      NO Asteroides
13         Asteroides
14         Asteroides
15         Asteroides
16         Asteroides
17         Asteroides
18         Asteroides
19         Asteroides
20         Asteroides
21      NO Asteroides
22      NO Asteroides
23      NO Asteroides
24      NO Asteroides
25      NO Asteroides
26      NO Asteroides
27      NO Asteroides
28      NO Asteroides
29      NO Asteroides
30      NO Asteroides
31      NO Asteroides
32      NO Asteroides
33      NO Asteroides
34      NO Asteroides
35      NO Asteroides
36      NO Asteroides
37      NO Asteroides
38      NO Asteroides
39      NO Asteroides
40      NO Asteroides
41      NO Asteroides
42      NO Asteroides
43      NO Asteroides
44      NO Asteroides
45      NO

In [246]:
Asteroides_relo=regresioncion_logistica(Asteroides,"Asteroides","NO Asteroides")

Matriz de Confusión: 
[[   0  319]
 [   4 1437]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.8164772727272728
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.8183371298405467
Exactitud del modelo:  0.8164772727272728
Sensibilidad del modelo:  0.9972241498959056
Puntaje F1 del modelo:  0.8989677822959025


In [247]:
Despegue= data_sin_nulos['mini juego'].replace(["Asteroides", "Estrellas", "Gusanos", "MiniGame_0", "MiniGame_1", "MiniGame_2", "MiniGame_3", "Restaurante", "animales y colores", "figuras y colores", "partes del cuerpo", "sonidos y animales"], "NO Despegue")
Despegue

0          Despegue
1          Despegue
2          Despegue
3          Despegue
4          Despegue
5          Despegue
6          Despegue
7          Despegue
8          Despegue
9          Despegue
10         Despegue
11         Despegue
12         Despegue
13      NO Despegue
14      NO Despegue
15      NO Despegue
16      NO Despegue
17      NO Despegue
18      NO Despegue
19      NO Despegue
20      NO Despegue
21      NO Despegue
22      NO Despegue
23      NO Despegue
24      NO Despegue
25      NO Despegue
26      NO Despegue
27      NO Despegue
28      NO Despegue
29      NO Despegue
30      NO Despegue
31      NO Despegue
32      NO Despegue
33      NO Despegue
34      NO Despegue
35      NO Despegue
36      NO Despegue
37      NO Despegue
38      NO Despegue
39      NO Despegue
40      NO Despegue
41      NO Despegue
42      NO Despegue
43      NO Despegue
44      NO Despegue
45      NO Despegue
46      NO Despegue
47      NO Despegue
48      NO Despegue
49      NO Despegue


In [248]:
Despegue_relo=regresioncion_logistica(Despegue,"Despegue","NO Despegue")

Matriz de Confusión: 
[[   0   65]
 [   0 1695]]
Modelo 0:
Precisión del modelo:  0.0
Exactitud del modelo:  0.9630681818181818
Sensibilidad del modelo:  0.0
Puntaje F1 del modelo:  0.0


Modelo 1:
Precisión del modelo:  0.9630681818181818
Exactitud del modelo:  0.9630681818181818
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  0.9811866859623734


c:\Users\B_3mi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Numero de Interacción

In [249]:
Max=data_sin_nulos['número de interacción'].max()
Min=data_sin_nulos['número de interacción'].min()
Limites= [Min, Max]
Limites

[np.float64(1.0), np.float64(256.0)]

In [250]:
intervalos=np.linspace(0.9, 256.1, 3)
print(intervalos)
categorias= ["Interacción Baja", "Interacción Alta"]

[  0.9 128.5 256.1]


In [251]:
pd.options.display.max_rows = None

In [252]:
internew=pd.cut(x= data_sin_nulos['número de interacción'], bins=intervalos, labels= categorias )
internew.head()

0    Interacción Baja
1    Interacción Baja
2    Interacción Baja
3    Interacción Baja
4    Interacción Baja
Name: número de interacción, dtype: category
Categories (2, object): ['Interacción Baja' < 'Interacción Alta']

In [253]:
internew_relo=regresioncion_logistica(internew,"Interacción Alta","Interacción Baja")

Matriz de Confusión: 
[[  32    0]
 [   0 1728]]
Modelo 0:
Precisión del modelo:  1.0
Exactitud del modelo:  1.0
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  1.0


Modelo 1:
Precisión del modelo:  1.0
Exactitud del modelo:  1.0
Sensibilidad del modelo:  1.0
Puntaje F1 del modelo:  1.0


C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\521905368.py:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dicotomica=dicotomica.replace(si,"0")
C:\Users\B_3mi\AppData\Local\Temp\ipykernel_16060\521905368.py:10: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  dicotomica=dicotomica.replace(no,"1")


In [254]:
data = data_sin_nulos.copy()

Usuario (5)

In [255]:
us = np.unique(data_sin_nulos['Usuario'])
us

array(['ADRIAN ', 'ALEIDA', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN',
       'CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE',
       'ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ',
       'JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO',
       'JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ',
       'MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL',
       'VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ',
       'jesus eduardo', 'nicolas', 'ramiro isai'], dtype=object)

In [256]:
ADRIAN= data_sin_nulos['Usuario'].replace(['ALEIDA', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ADRIAN")
ADRIAN.head()

0    NO ADRIAN
1    NO ADRIAN
2    NO ADRIAN
3    NO ADRIAN
4    NO ADRIAN
Name: Usuario, dtype: object

In [257]:
ADRIAN_relo=regresioncion_logistica(ADRIAN,"ADRIAN","NO ADRIAN")

ValueError: Input y contains NaN.

In [ ]:
ALEIDA= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ARLETT', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ALEIDA")
ALEIDA.head()

In [ ]:
ALEIDA_relo=regresioncion_logistica(ALEIDA,"ALEIDA","NO ALEIDA")

In [ ]:
ARLETT= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ASHLEY ', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ARLETT")
ARLETT.head()

In [ ]:
ARLETT_relo=regresioncion_logistica(ARLETT,"ARLETT","NO ARLETT")

In [ ]:
ASHLEY= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ARLETT', 'AUSTIN', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO ASHLEY")
ASHLEY.head()

In [ ]:
ASHLEY_relo=regresioncion_logistica(ASHLEY,"ASHLEY","NO ASHLEY")

In [ ]:
AUSTIN= data_sin_nulos['Usuario'].replace(['ADRIAN', 'ALEIDA', 'ARLETT', 'ASHLEY ', 'BENJAMIN','CARLOS ABEL', 'CARLOS ENRIQUE', 'CONCEPCION', 'DENISSE','ERICK OSVALDO', 'ESMERALDA', 'IKER BENJAMIN', 'INGRID', 'IRVING ','JESUS ALEJANDRO', 'JOSE IAN', 'JOSE IGNACIO TADEO','JOSE JAVIER ', 'JOSHUA', 'KYTZIA', 'LEONARDO', 'LEONARDO ','MA DEL ROSARIO ', 'NICOLAS |', 'RENE ', 'SERGIO ANGEL','VALENTIN', 'YAEL DAVID', 'YEREMI YAZMIN ', 'erick ','jesus eduardo', 'nicolas', 'ramiro isai'], "NO AUSTIN")
AUSTIN.head()

In [ ]:
AUSTIN_relo=regresioncion_logistica(AUSTIN,"AUSTIN","NO AUSTIN")

## Realizar una tabla de todos los coeficientes de precisión, exactitud y sensibilidad 

Botón correcto

In [258]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Botón correcto 0.0', 'Botón correcto 1.0'],
    'Precisión': [precision1_0, precision1],
    'Exactitud': [exactitud1_0, exactitud1],
    'Sensibilidad (Recall)': [sensibilidad1_0, sensibilidad1],
    'Puntaje F1': [puntajef1_1_0, puntajef1_1]
})

# Mostrar tabla
print("Métricas de Botón Correcto: \n", tabla_metricas)

Métricas de Botón Correcto: 
                 Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Botón correcto 0.0   0.746667   0.655781               0.498728    0.598017
1  Botón correcto 1.0   0.608350   0.655781               0.821477    0.699029


Juego

In [259]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Botón correcto 0.0', 'Botón correcto 1.0'],
    'Precisión': [precision2_0, precision2],
    'Exactitud': [exactitud2_0, exactitud2],
    'Sensibilidad (Recall)': [sensibilidad2_0, sensibilidad2],
    'Puntaje F1': [puntajef1_2_0, puntajef1_2]
})

# Mostrar tabla
print("Métricas de Juego: \n",tabla_metricas)

Métricas de Juego: 
                 Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Botón correcto 0.0   0.726214   0.651208               0.487614    0.583463
1  Botón correcto 1.0   0.613189   0.651208               0.815445    0.700000


Auto push

In [260]:
tabla_metricas = pd.DataFrame({
    'Clase': ['Botón correcto 0.0', 'Botón correcto 1.0'],
    'Precisión': [precision3_0, precision3],
    'Exactitud': [exactitud3_0, exactitud3],
    'Sensibilidad (Recall)': [sensibilidad3_0, sensibilidad3],
    'Puntaje F1': [puntajef1_3_0, puntajef1_3]
})

# Mostrar tabla
print("Métricas de Auto Push: \n",tabla_metricas)

Métricas de Auto Push: 
                 Clase  Precisión  Exactitud  Sensibilidad (Recall)  Puntaje F1
0  Botón correcto 0.0   0.743542   0.674069               0.528178    0.598017
1  Botón correcto 1.0   0.635996   0.674069               0.819010    0.699029
